In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=f39a2e80ef01f85fe678fe27920a0e833b5a1d41a3d452fb5de38d0a5749e5fc
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
!pip install dnspython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 4.2 MB/s eta 0:00:00


In [47]:
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import (
    col, count, when, isnan, split, size,
    array_min, array_max, array_distinct, to_timestamp,
    spark_partition_id
)

import pandas as pd
pd.set_option('display.max_columns', None)

import psycopg2

from tqdm import tqdm

In [7]:
spark = SparkSession \
    .builder \
    .config("spark.driver.memory", "10g") \
    .config("spark.jars", "postgresql-42.6.0.jar") \
    .appName("Yelp Review Sense by Yash Raizada") \
    .getOrCreate()

In [9]:
!tar -xvzf /content/yelp_dataset.tgz

Dataset_User_Agreement.pdf
yelp_academic_dataset_business.json
yelp_academic_dataset_checkin.json
yelp_academic_dataset_review.json
yelp_academic_dataset_tip.json
yelp_academic_dataset_user.json


## Prepare User Dataset

<img src="https://github.com/yashraizada/yelp-review-sense/blob/main/images/Yelp%20User%20Profile.png?raw=true"/>

In [10]:
df_user = spark.read.json('yelp_academic_dataset_user.json')

In [11]:
df_user.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [12]:
df_user.describe().toPandas()

,summary,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,count,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897,1987897
1,mean,3.63049415035087,2.8293080577112395,0.13364927860950543,2.8293080577112395,1.8070724992290847,0.06390723463036566,0.29226313033321144,1.4431346292086562,1.226859339291724,3.0118411567601338,0.17931764070271247,1.056448095650831,23.792913817969442,2017.220760233918,1.4657404282012598,None,16.970536199813168,NaN,23.394409267683386,42.29633527290398,None,None
2,stddev,1.1833369995975145,96.66385446471259,11.356823097797161,96.66385446471259,73.60184094967366,10.04362659411937,12.824667119930178,60.16790049259038,95.1575129716489,119.38980389840354,15.155253377853501,32.17972805346817,565.3512954713351,4.09363504648036,18.13075272385579,None,407.8034374841018,NaN,82.56699161797889,641.4805967755902,None,None
3,min,1.0,0,0,0,0,0,0,0,0,0,0,0,0,,0,"---2PmXbF47D870stH1jqA, AgkO3LoVmmW6NyMZdHQmSw...",0,Chris,0,0,---1lKK3aKOuomHnwAkAow,2004-10-12 08:46:11
4,max,5.0,49967,13654,49967,25784,12669,13501,59031,82630,101097,14180,15934,199878,2021,12497,"zzzOOwSYzuTPvsafSCzDPw, djDWZw59RlGHuXKw20kyzQ...",185823,Ｊｏａｎｎｅ,17473,206296,zzzUFM4HFe0SFG0bPjntQA,2022-01-19 17:15:47


In [13]:
df_user.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_user.columns]).show()

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+-----+----+-------+-----+----+------------+------+-------+-------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer|cool|elite|fans|friends|funny|name|review_count|useful|user_id|yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+-----+----+-------+-----+----+------------+------+-------+-------------+
|            0|              0|              0|               0|             0|              0|              0|              0|                0|               0|       

## Data Preparation

### 1. Consolidating Compliment Columns
The compliment columns, such as "compliment_cool," "compliment_cute," and others, contain information about various types of compliments that users have received or given. Consolidating multiple compliment columns into one will simplify the dataset, enhance interpretability, and reduce redundancy, facilitating a more comprehensive understanding of user engagement.

### 2. Combining Interaction Columns
For the same reasons as consolidating the compliments columns, combining the 'useful,' 'funny,' and 'cool' columns into 'total_interactions'

### 3. Extracting Information from "Elite" and "Friends" Columns
The "elite" and "friends" columns are of string data type containing multiple values (years and user IDs, respectively) delimited by a comma. They are not inherently informative for meaningful analysis. To enable a more robust analysis, we can extract information such as the number of elite years, number of friends, etc.

### 4. Dropping Redundant Columns
The "name" column is redundant and needs to be dropped.

### 5. Optimizing Data Types
The data types for the 'fans,' 'review_count,' 'total_compliments,' and 'total_interactions' columns are not the most efficient representations for the data they contain. Changing to Integer as it better aligns with the nature of the data and optimizes memory usage.

In [14]:
# 1. Consolidating Compliment Columns
compliment_columns = [
    'compliment_cool', 'compliment_cute', 'compliment_funny','compliment_hot', 'compliment_list', 'compliment_more',
    'compliment_note', 'compliment_photos', 'compliment_plain','compliment_profile', 'compliment_writer'
]

df_user = df_user.withColumn('total_compliments', sum(col(col_name) for col_name in compliment_columns))
df_user = df_user.drop(*compliment_columns)

In [15]:
# 2. Combining Interaction Columns
interaction_columns = ['useful', 'funny', 'cool']

df_user = df_user.withColumn('total_interactions', sum(col(col_name) for col_name in interaction_columns))
df_user = df_user.drop(*interaction_columns)

In [16]:
# 3. Extracting Information from "Elite" and "Friends" Columns
df_user = df_user.withColumn('elite_years_array', split(df_user['elite'], ","))
df_user = df_user.withColumn('elite_years_count', when(df_user['elite'] != "", size(df_user['elite_years_array'])).otherwise(0)) \
                 .withColumn('elite_min_year', when(df_user['elite'] != "", array_min(df_user['elite_years_array']).cast('integer')).otherwise(0)) \
                 .withColumn('elite_max_year', when(df_user['elite'] != "", array_max(df_user['elite_years_array']).cast('integer')).otherwise(0))

df_user = df_user.withColumn('friends_list', split(df_user['friends'], ","))
df_user = df_user.withColumn('friends_count', size(array_distinct(df_user['friends_list'])))

columns_to_drop = ['elite', 'elite_years_array', 'friends', 'friends_list']
df_user = df_user.drop(*columns_to_drop)

In [17]:
# 4. Dropping Redundant Columns
columns_to_drop = ['name']
df_user = df_user.drop(*columns_to_drop)

In [18]:
# 5. Optimizing Data Types
int_columns = ['fans', 'review_count', 'total_compliments', 'total_interactions']

for col_name in int_columns:
    df_user = df_user.withColumn(col_name, df_user[col_name].cast(IntegerType()))

df_user = df_user.withColumn('yelping_since', to_timestamp(df_user['yelping_since'], 'yyyy-MM-dd HH:mm:ss'))

In [19]:
df_user.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- fans: integer (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: timestamp (nullable = true)
 |-- total_compliments: integer (nullable = true)
 |-- total_interactions: integer (nullable = true)
 |-- elite_years_count: integer (nullable = false)
 |-- elite_min_year: integer (nullable = true)
 |-- elite_max_year: integer (nullable = true)
 |-- friends_count: integer (nullable = false)



In [20]:
column_order = [
    "user_id",
    "yelping_since",
    "review_count",
    "average_stars",
    "fans",
    "friends_count",
    "total_interactions",
    "total_compliments",
    "elite_years_count",
    "elite_min_year",
    "elite_max_year"
]

df_user = df_user.select(column_order)
df_user.show(5)

+--------------------+-------------------+------------+-------------+----+-------------+------------------+-----------------+-----------------+--------------+--------------+
|             user_id|      yelping_since|review_count|average_stars|fans|friends_count|total_interactions|total_compliments|elite_years_count|elite_min_year|elite_max_year|
+--------------------+-------------------+------------+-------------+----+-------------+------------------+-----------------+-----------------+--------------+--------------+
|qVc8ODYU5SZjKXVBg...|2007-01-25 16:47:26|         585|         3.91| 267|        14995|             14470|             2873|                1|          2007|          2007|
|j14WgRoU_-2ZE1aw1...|2009-01-25 04:35:42|        4333|         3.74|3138|         4646|             83438|            20631|               14|            20|          2021|
|2WnXYQFK0hXEoTxPt...|2008-07-25 10:41:00|         665|         3.32|  52|          381|              4099|              585|     

## Prepare Business Dataset

<img src="https://github.com/yashraizada/yelp-review-sense/blob/main/images/Yelp%20Business%20Profile.png?raw=true"/>

In [21]:
df_business = spark.read.json('yelp_academic_dataset_business.json')

In [22]:
df_business.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [23]:
# Dropping 'attributes' and 'hours' cppulmns to help streamline the dataset and focus on the most relevant variables

struct_columns_to_drop = ['attributes', 'hours']
df_business = df_business.drop(*struct_columns_to_drop)

In [24]:
df_business.describe().toPandas()

,summary,address,business_id,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,count,150346,150346,150243,150346,150346,150346,150346,150346,150346,150346,150346,150346
1,mean,7369.333333333333,None,None,None,0.7961502135075094,36.67115006414526,-89.35733948971448,1252.4,45177.81755426108,44.86656113232144,3.5967235576603303,None
2,stddev,8738.777641447725,None,None,None,0.402859939090063,5.872758917014039,14.918501679930593,811.1275005954503,26395.882085856578,121.12013570117037,0.9744207509201411,None
3,min,,---kPU91CF4Lq2-WlRu9Lw,"3D Printing, Local Services, Hobby Shops, Shop...",AB Edmonton,0,27.555127,-120.095137,Grow Academy,,5,1.0,AB
4,max,​185 E State St,zzyx5x0Z7xXWWvWnZFuxlQ,"Zoos, Tours, Arts & Entertainment, Hotels & Tr...",​Lithia,1,53.6791969,-73.2004570502,​​Transformational Abdominal Massage by Jada D...,T9E 0V3,7568,5.0,XMS


In [25]:
df_business.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_business.columns]).show()

+-------+-----------+----------+----+-------+--------+---------+----+-----------+------------+-----+-----+
|address|business_id|categories|city|is_open|latitude|longitude|name|postal_code|review_count|stars|state|
+-------+-----------+----------+----+-------+--------+---------+----+-----------+------------+-----+-----+
|      0|          0|       103|   0|      0|       0|        0|   0|          0|           0|    0|    0|
+-------+-----------+----------+----+-------+--------+---------+----+-----------+------------+-----+-----+



## Data Preparation

### 1. Dropping Redundant Columns
The "address", "attributes", "is_open", and "hours" columns are redundant and needs to be dropped. The "categories" column is not inherently informative for meaningful analysis.

### 2. Optimizing Data Types
The data types for the "latitude," "longitude," and "stars" columns are not the most efficient representations for the data they contain. Float aligns better. Similarly, "review_count" is converted to Integers.

In [26]:
# 1. Dropping Redundant Columns
columns_to_drop = ['address', 'categories', 'is_open']
df_business = df_business.drop(*columns_to_drop)

In [27]:
# 2. Optimizing Data Types

# Convert latitude, longitude, and stars to float
df_business = df_business.withColumn('latitude', df_business['latitude'].cast(FloatType()))
df_business = df_business.withColumn('longitude', df_business['longitude'].cast(FloatType()))
df_business = df_business.withColumn('stars', df_business['stars'].cast(FloatType()))

# Convert review_count to integer
df_business = df_business.withColumn('review_count', df_business['review_count'].cast(IntegerType()))

In [28]:
df_business.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- stars: float (nullable = true)
 |-- state: string (nullable = true)



In [29]:
column_order = [
    "business_id",
    "name",
    "city",
    "state",
    "postal_code",
    "latitude",
    "longitude",
    "stars",
    "review_count"
]

df_business = df_business.select(column_order)
df_business.show(5)

+--------------------+--------------------+-------------+-----+-----------+---------+-----------+-----+------------+
|         business_id|                name|         city|state|postal_code| latitude|  longitude|stars|review_count|
+--------------------+--------------------+-------------+-----+-----------+---------+-----------+-----+------------+
|Pns2l4eNsfO8kk83d...|Abby Rappoport, L...|Santa Barbara|   CA|      93101|34.426678|  -119.7112|  5.0|           7|
|mpf3x-BjTdTEA3yCZ...|       The UPS Store|       Affton|   MO|      63123|38.551125|  -90.33569|  3.0|          15|
|tUFrWirKiKi_TAnsV...|              Target|       Tucson|   AZ|      85711|32.223236|-110.880455|  3.5|          22|
|MTSW4McQd7CbVtyjq...|  St Honore Pastries| Philadelphia|   PA|      19107|39.955505|  -75.15556|  4.0|          80|
|mWMc6_wTdE0EUBKIG...|Perkiomen Valley ...|   Green Lane|   PA|      18054|40.338184|  -75.47166|  4.5|          13|
+--------------------+--------------------+-------------+-----+-

## Prepare Review Dataset

<img src="https://github.com/yashraizada/yelp-review-sense/blob/main/images/Sample%20Review.png?raw=true"/>

In [30]:
df_review = spark.read.json('yelp_academic_dataset_review.json')

In [31]:
df_review.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [32]:
df_review.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_review.columns]).show()

+-----------+----+----+-----+---------+-----+----+------+-------+
|business_id|cool|date|funny|review_id|stars|text|useful|user_id|
+-----------+----+----+-----+---------+-----+----+------+-------+
|          0|   0|   0|    0|        0|    0|   0|     0|      0|
+-----------+----+----+-----+---------+-----+----+------+-------+



In [33]:
# Consolidating multiple interactions columns

columns_to_combine = ['useful', 'funny', 'cool']
df_review = df_review.withColumn('total_interaction', sum(col(col_name) for col_name in columns_to_combine))
df_review = df_review.drop(*columns_to_combine)

In [34]:
# Correcting data types and optimizing memory usage

df_review = df_review.withColumn('total_interaction', df_review['total_interaction'].cast(IntegerType()))
df_review = df_review.withColumn('date', to_timestamp(df_review['date'], 'yyyy-MM-dd HH:mm:ss'))

In [35]:
column_order = [
    'review_id',
    'business_id',
    'user_id',
    'date',
    'stars',
    'text',
    'total_interaction'
]

df_review = df_review.select(column_order)
df_review.show(5)

+--------------------+--------------------+--------------------+-------------------+-----+--------------------+-----------------+
|           review_id|         business_id|             user_id|               date|stars|                text|total_interaction|
+--------------------+--------------------+--------------------+-------------------+-----+--------------------+-----------------+
|KU_O5udG6zpxOg-Vc...|XQfwVwDr-v0ZS3_Cb...|mh_-eMZ6K5RLWhZyI...|2018-07-07 22:09:11|  3.0|If you decide to ...|                0|
|BiTunyQ73aT9WBnpR...|7ATYjTIgM3jUlt4UM...|OyoGAe7OKpv6SyGZT...|2012-01-03 15:28:18|  5.0|I've taken a lot ...|                2|
|saUsX_uimxRlCVr67...|YjUWPpI6HXG530lwP...|8g_iMtfSiwikVnbP2...|2014-02-05 20:30:30|  3.0|Family diner. Had...|                0|
|AqPFMleE6RsU23_au...|kxX2SOes4o-D3ZQBk...|_7bHUi9Uuf5__HHc_...|2015-01-04 00:01:03|  5.0|Wow!  Yummy, diff...|                2|
|Sx8TMOWLNuJBWer-0...|e4Vwtrqf-wpJfwesg...|bcjbaE6dDog4jkNY9...|2017-01-14 20:54:15|  4.0|

## Prepare Check-in Dataset

In [36]:
df_checkin = spark.read.json('yelp_academic_dataset_checkin.json')

In [37]:
df_checkin.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)



In [38]:
df_checkin.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_checkin.columns]).show()

+-----------+----+
|business_id|date|
+-----------+----+
|          0|   0|
+-----------+----+



In [39]:
# Extract the oldest and newest date of check-ins

df_checkin = df_checkin.withColumn('date_list', split(df_checkin['date'], ','))
df_checkin = df_checkin.withColumn('checkin_count', size(df_checkin['date_list']))
df_checkin = df_checkin.withColumn('checkin_date_min', to_timestamp(array_min(df_checkin['date_list'])))
df_checkin = df_checkin.withColumn('checkin_date_max', to_timestamp(array_max(df_checkin['date_list'])))

columns_to_drop = ['date', 'date_list']
df_checkin = df_checkin.drop(*columns_to_drop)
df_checkin.show(5)

+--------------------+-------------+-------------------+-------------------+
|         business_id|checkin_count|   checkin_date_min|   checkin_date_max|
+--------------------+-------------+-------------------+-------------------+
|---kPU91CF4Lq2-Wl...|           11|2020-06-02 22:18:06|2020-03-13 21:10:56|
|--0iUa4sNDFiZFrAd...|           10|2011-05-04 23:08:15|2010-09-13 21:43:09|
|--30_8IhuyMHbSOcN...|            2|2014-08-13 23:20:22|2013-06-14 23:29:17|
|--7PUidqRWpRSpXeb...|           10|2011-07-28 02:46:10|2011-02-15 17:12:00|
|--7jw19RH9JKXgFoh...|           26|2014-04-28 21:04:46|2014-04-21 20:42:11|
+--------------------+-------------+-------------------+-------------------+
only showing top 5 rows



## Prepare Tip Dataset

In [40]:
df_tip = spark.read.json('yelp_academic_dataset_tip.json')

In [41]:
df_tip.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- compliment_count: long (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)



In [42]:
df_tip.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_tip.columns]).show()

+-----------+----------------+----+----+-------+
|business_id|compliment_count|date|text|user_id|
+-----------+----------------+----+----+-------+
|          0|               0|   0|   0|      0|
+-----------+----------------+----+----+-------+



In [43]:
df_tip = df_tip.withColumn("compliment_count", df_tip["compliment_count"].cast(IntegerType()))
df_tip = df_tip.withColumn('date', to_timestamp(df_tip['date'], 'yyyy-MM-dd HH:mm:ss'))

In [44]:
df_tip.show(5)

+--------------------+----------------+-------------------+--------------------+--------------------+
|         business_id|compliment_count|               date|                text|             user_id|
+--------------------+----------------+-------------------+--------------------+--------------------+
|3uLgwr0qeCNMjKenH...|               0|2012-05-18 02:17:21|Avengers time wit...|AGNUgVwnZUey3gcPC...|
|QoezRbYQncpRqyrLH...|               0|2013-02-05 18:35:10|They have lots of...|NBN4MgHP9D3cw--Sn...|
|MYoRNLb5chwjQe3c_...|               0|2013-08-18 00:56:08|It's open even wh...|-copOvldyKh1qr-vz...|
|hV-bABTK-glh5wj31...|               0|2017-06-27 23:05:38|Very decent fried...|FjMQVZjSqY8syIO-5...|
|_uN0OudeJ3Zl_tf6n...|               0|2012-10-06 19:43:09|Appetizers.. plat...|ld0AperBXk1h6Ubqm...|
+--------------------+----------------+-------------------+--------------------+--------------------+
only showing top 5 rows



## Merge Dataset

In [45]:
df = df_review.select(
    col("review_id"),
    col("user_id"),
    col("business_id"),
    col("date").alias("review_date"),
    col("stars").alias("review_stars"),
    col("text").alias("review_text"),
    col("total_interaction").alias("review_total_interaction")
)

df = df.join(
    df_user.select(
        col("user_id"),
        col("yelping_since").alias("user_yelping_since"),
        col("review_count").alias("user_review_count"),
        col("average_stars").alias("user_average_stars"),
        col("fans").alias("user_fans"),
        col("friends_count").alias("user_friends_count"),
        col("total_interactions").alias("user_total_interactions"),
        col("total_compliments").alias("user_total_compliments"),
        col("elite_years_count").alias("user_elite_years_count"),
        col("elite_min_year").alias("user_elite_min_year"),
        col("elite_max_year").alias("user_elite_max_year")
    ),
    on="user_id",
    how="left"
)

df = df.join(
    df_business.select(
        col("business_id"),
        col("name").alias("biz_name"),
        col("city").alias("biz_city"),
        col("state").alias("biz_state"),
        col("postal_code").alias("biz_postal_code"),
        col("latitude").alias("biz_latitude"),
        col("longitude").alias("biz_longitude"),
        col("stars").alias("biz_stars"),
        col("review_count").alias("biz_review_count")
    ),
    on="business_id",
    how="left"
)

df = df.join(
    df_checkin.select(
        col("business_id"),
        col("checkin_count").alias("checkin_count"),
        col("checkin_date_min").alias("checkin_date_min"),
        col("checkin_date_max").alias("checkin_date_max")
    ),
    on="business_id",
    how="left"
)

df.show()

+--------------------+--------------------+--------------------+-------------------+------------+--------------------+------------------------+-------------------+-----------------+------------------+---------+------------------+-----------------------+----------------------+----------------------+-------------------+-------------------+--------------------+----------------+---------+---------------+------------+-------------+---------+----------------+-------------+-------------------+-------------------+
|         business_id|             user_id|           review_id|        review_date|review_stars|         review_text|review_total_interaction| user_yelping_since|user_review_count|user_average_stars|user_fans|user_friends_count|user_total_interactions|user_total_compliments|user_elite_years_count|user_elite_min_year|user_elite_max_year|            biz_name|        biz_city|biz_state|biz_postal_code|biz_latitude|biz_longitude|biz_stars|biz_review_count|checkin_count|   checkin_date_m

In [53]:
df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- review_stars: double (nullable = true)
 |-- review_text: string (nullable = true)
 |-- review_total_interaction: integer (nullable = true)
 |-- user_yelping_since: timestamp (nullable = true)
 |-- user_review_count: integer (nullable = true)
 |-- user_average_stars: double (nullable = true)
 |-- user_fans: integer (nullable = true)
 |-- user_friends_count: integer (nullable = true)
 |-- user_total_interactions: integer (nullable = true)
 |-- user_total_compliments: integer (nullable = true)
 |-- user_elite_years_count: integer (nullable = true)
 |-- user_elite_min_year: integer (nullable = true)
 |-- user_elite_max_year: integer (nullable = true)
 |-- biz_name: string (nullable = true)
 |-- biz_city: string (nullable = true)
 |-- biz_state: string (nullable = true)
 |-- biz_postal_code: string (nullable = true

In [ ]:
print("Shape of master df:", (df.count(), len(df.columns)))

Shape of master df: (6990280, 28)


## Data Export to AWS

In [ ]:
# Define the JDBC connection details
jdbc_url = "jdbc:postgresql://intentionally-removed.amazonaws.com/yelpreviewsense"
db_table = ""
db_user = "postgres"
db_password = ""

# Specify the DataFrame write options
write_options = {
    "url": jdbc_url,
    "driver": "org.postgresql.Driver",
    "dbtable": db_table,
    "user": db_user,
    "password": db_password,
}

# Write the DataFrame to the PostgreSQL database
df.write.format("jdbc").options(**write_options).mode("overwrite").save()